# 参数初始化

## 一.使用Numpy来初始化

In [1]:
import numpy as np
import torch
from torch import nn

### 1.Sequential模型中使用Numpy初始化

In [2]:
#定义一个Sequential模型
net1=nn.Sequential(nn.Linear(30,40),
                  nn.ReLU(),
                  nn.Linear(40,50),
                  nn.ReLU(),
                  nn.Linear(50,10))

In [36]:
#访问第一层参数
w1=net1[0].weight #注意weight是属性而不是方法,只有Linear层等包括参数的才有weight属性,而nn.ReLU()
b1=net1[0].bias

In [37]:
print(w1)
print(w1.shape)
print(net1[2].weight.shape)#发现了torch中的参数矩阵的shape和我们印象中的反着来,Linear中设置为(30,40),实际上矩阵shape为(40,30)

Parameter containing:
tensor([[-0.1798, -0.1446, -0.1128,  ...,  0.1068,  0.1280,  0.0069],
        [ 0.1491,  0.2916, -0.2336,  ...,  0.2075,  0.0388, -0.1098],
        [ 0.2119, -0.2025,  0.2661,  ..., -0.0012, -0.0089, -0.1479],
        ...,
        [ 0.2141,  0.2745,  0.0413,  ...,  0.2463,  0.1104,  0.2462],
        [ 0.2500, -0.1446,  0.1345,  ..., -0.0570,  0.1312,  0.0334],
        [-0.0577, -0.0456,  0.1500,  ..., -0.0247, -0.2277, -0.0339]],
       requires_grad=True)
torch.Size([40, 30])
torch.Size([50, 40])


w1是Parameter,本质是一个Variable,可以访问.data属性得到其中的数据,然后直接定义一个新的Tensor对其进行替换.这也是能够使用numpy初始化模型中参数的原因之一.
#### 定义一个Tensor直接对其进行替换

In [11]:
net1[0].weight.data=torch.Tensor(np.random.uniform(3,5,size=(40,30)))#注意是(40,30)而(30,40)

print(net1[0].weight)

Parameter containing:
tensor([[4.4739, 3.5678, 3.4574,  ..., 4.0851, 3.9073, 3.8313],
        [4.8488, 3.0264, 3.8790,  ..., 3.3691, 3.1821, 4.3583],
        [3.9068, 3.6489, 4.4799,  ..., 3.8133, 3.0004, 3.6986],
        ...,
        [3.7571, 3.8348, 4.3450,  ..., 3.9596, 4.9824, 4.7106],
        [4.4689, 4.5710, 3.0487,  ..., 4.8061, 4.9255, 4.5129],
        [4.5979, 4.6713, 4.8290,  ..., 3.3304, 3.5293, 4.4833]],
       requires_grad=True)


可以看到参数的值已经被改变了,通过numpy的初始化定义Tensor,然后替换,达到自定义初始化的目的.

如果某一层需要我们手动去改,可以通过此种方式.

但是大多数情况需要相同类型的层都需要相同的初始化方式,我们可以直接用循环来访问每一层的参数,比如:

In [17]:
for layer in net1:
    if isinstance(layer,nn.Linear):# 判断这一层是全连接层
        param_shape=layer.weight.shape #得到形状.weight,bias,data,shape等都是属性,没有括号
        layer.weight.data=torch.Tensor(np.random.normal(0,0.5,size=param_shape)) #定义均值为0,方差为0.5的正态分布

#### 扩展:`isinstance(object1,type)`:判断类型是否是目标类型,这是python中的函数.
`instance` denotes 实例,情况. `instance of` denotes 某类的实例.

In [15]:
print(type(net1[0]))
print(isinstance(5,int))

<class 'torch.nn.modules.linear.Linear'>
True


### 2.Module中使用Numpy初始化
和Sequential中类似,如果对某一层的参数进行自定义初始化,可以直接访问替换.

唯一的不同之处是,如果用循环的方式访问,需要介绍两个属性方法,children()和modules,下面我们举例来说明

In [18]:
class sim_net(nn.Module):
    def __init__(self):
        super(sim_net,self).__init__()
        self.l1=nn.Sequential(nn.Linear(30,40),
                             nn.ReLU())
        self.l1[0].weight.data=torch.randn(40,30)#直接对某一层进行初始化
        self.l2=nn.Sequential(nn.Linear(40,50),
                             nn.ReLU())
        self.l3=nn.Sequential(nn.Linear(50,10),
                             nn.ReLU())
    def forward(self,x):
        x=self.l1(x)
        x=self.l2(x)
        x=self.l3(x)
        return x

我们发现,Module中定义模型还可以嵌套nn.Sequential()

In [19]:
net2=sim_net()

#### 1. 访问model.children()

In [21]:
for i in net2.children():
    print(i)

Sequential(
  (0): Linear(in_features=30, out_features=40, bias=True)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=40, out_features=50, bias=True)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=50, out_features=10, bias=True)
  (1): ReLU()
)


我们发现,children就是在Module中嵌套的每个Sequential.

#### 2. 访问model.modules()

In [25]:
for i in net2.modules():
    print(i)
    print('--------------------------------------------------------------')

sim_net(
  (l1): Sequential(
    (0): Linear(in_features=30, out_features=40, bias=True)
    (1): ReLU()
  )
  (l2): Sequential(
    (0): Linear(in_features=40, out_features=50, bias=True)
    (1): ReLU()
  )
  (l3): Sequential(
    (0): Linear(in_features=50, out_features=10, bias=True)
    (1): ReLU()
  )
)
--------------------------------------------------------------
Sequential(
  (0): Linear(in_features=30, out_features=40, bias=True)
  (1): ReLU()
)
--------------------------------------------------------------
Linear(in_features=30, out_features=40, bias=True)
--------------------------------------------------------------
ReLU()
--------------------------------------------------------------
Sequential(
  (0): Linear(in_features=40, out_features=50, bias=True)
  (1): ReLU()
)
--------------------------------------------------------------
Linear(in_features=40, out_features=50, bias=True)
--------------------------------------------------------------
ReLU()
-----------------------

#### 3.`model.children()`和`model.modules()`的区别:
我们发现,`model.children()`访问了每一个`Sequential()`,没有往下展开.而`model.modules()`不仅访问了每个`Sequential()`还对其进行了展开.

#### 4. Module模型基于model.modules()进行初始化

In [27]:
for layer in net2.modules():
    if isinstance(layer,nn.Linear):
        param_shape=layer.weight.shape
        layer.weight.data=torch.from_numpy(np.random.normal(0,0.5,size=param_shape))#替换

## 二. 使用`torch.nn.init`来初始化
初始化的模型也是在nn中,我们可以直接对Tensor进行操作从而初始化,操作仍然在Tensor上,下面我们举例说明.

In [28]:
from torch.nn import init
print(net1[0].weight)

Parameter containing:
tensor([[-0.4898,  0.2482, -0.1636,  ...,  0.8408,  0.0324, -0.1102],
        [ 1.1129, -0.0292, -0.2416,  ...,  0.1716, -0.9831,  0.6827],
        [-0.1292, -0.5616, -0.7552,  ...,  0.2098,  0.0949, -0.4321],
        ...,
        [ 0.3916,  0.0053,  0.1710,  ..., -0.0265, -0.2259,  0.3010],
        [-0.3619,  0.1388,  0.1740,  ..., -0.3977,  1.1801, -0.2805],
        [ 0.0806,  0.4990,  0.2464,  ...,  0.2873, -0.8950,  0.1862]],
       requires_grad=True)


### 原地操作:`init.initFunc(model[idx].weight)`: `init.axvier(net1[0].weight)`

In [45]:
init.xavier_uniform_(net1[0].weight)

Parameter containing:
tensor([[-0.2396, -0.0449,  0.0152,  ...,  0.2764,  0.0389, -0.1339],
        [ 0.1418, -0.1649,  0.2751,  ...,  0.1006,  0.0465,  0.0545],
        [ 0.1993,  0.2090,  0.1090,  ...,  0.1255, -0.0166,  0.2398],
        ...,
        [ 0.2361, -0.2697, -0.0483,  ...,  0.2866,  0.1582,  0.2351],
        [-0.0586, -0.2322, -0.2707,  ..., -0.0372, -0.0507, -0.0188],
        [-0.2382, -0.2509,  0.2771,  ...,  0.0756, -0.0319, -0.1729]],
       requires_grad=True)

In [30]:
print(net1[0].weight)

Parameter containing:
tensor([[-0.1798, -0.1446, -0.1128,  ...,  0.1068,  0.1280,  0.0069],
        [ 0.1491,  0.2916, -0.2336,  ...,  0.2075,  0.0388, -0.1098],
        [ 0.2119, -0.2025,  0.2661,  ..., -0.0012, -0.0089, -0.1479],
        ...,
        [ 0.2141,  0.2745,  0.0413,  ...,  0.2463,  0.1104,  0.2462],
        [ 0.2500, -0.1446,  0.1345,  ..., -0.0570,  0.1312,  0.0334],
        [-0.0577, -0.0456,  0.1500,  ..., -0.0247, -0.2277, -0.0339]],
       requires_grad=True)


torch.nn.init提供了更多的内置初始化方式,xavier只是其中之一.

## 小结
1. `torch.nn.init.axvier_uniform_(Tensor)`:对Tensor进行一种初始化.注意新版的torch是axvier_uniform_, axvier_uniform已经被deprecated

In [44]:
from torch.autograd import Variable
tensor1=Variable(torch.zeros(2,3))
print(tensor1)
init.xavier_uniform_(tensor1)
print(tensor1)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[ 0.2132,  0.2888, -0.9336],
        [ 0.0862, -0.5392,  0.4674]])
